In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import RegressionResults

# 線形回帰を実施する関数
def perform_linear_regression(data, explanatory_vars, dependent_var, min_data_count):
    results = {}
    plots = {}

    for date in data['Date'].unique():
        data_date = data[data['Date'] == date]
        for sector in data['Sector'].unique():
            data_sector = data_date[data_date['Sector'] == sector]

            if len(data_sector) >= min_data_count:
                X = data_sector[explanatory_vars]
                y = data_sector[dependent_var]
                X = sm.add_constant(X)  # 定数項の追加

                model = sm.OLS(y, X).fit()
                results[(date, sector)] = model
                
                # サブプロットの作成
                fig, ax = plt.subplots(figsize=(6, 4))
                ax.scatter(X[explanatory_vars[0]], y, color='blue')
                ax.plot(X[explanatory_vars[0]], model.predict(X), color='red')
                ax.set_title(f"{sector} - {date.strftime('%Y-%m')}")
                ax.set_xlabel(explanatory_vars[0])
                ax.set_ylabel(dependent_var)
                plots[(date, sector)] = fig

    return results, plots

# 説明変数と被説明変数を指定
explanatory_vars = ['Credit_Rating', 'Credit_Spread', 'Remaining_Years', 'Coupon_Rate', 'Issue_Amount']
dependent_var = 'Total_Return'

# 線形回帰を実行（各セクターで最低50のデータポイントが必要）
results, plots = perform_linear_regression(sample_data, explanatory_vars, dependent_var, min_data_count=50)

# 最初の結果のサマリーを表示（例）
first_key = list(results.keys())[0]
print(results[first_key].summary())


In [ ]:
# 線形回帰を実施する関数（修正版）
def perform_linear_regression_fixed(data, explanatory_vars, dependent_var, min_data_count):
    results = {}

    for date in data['Date'].unique():
        data_date = data[data['Date'] == date]
        for sector in data['Sector'].unique():
            data_sector = data_date[data_date['Sector'] == sector]

            if len(data_sector) >= min_data_count:
                X = data_sector[explanatory_vars]
                y = data_sector[dependent_var]
                X = sm.add_constant(X)  # 定数項の追加

                model = sm.OLS(y, X).fit()
                results[(date, sector)] = model

    return results

# 線形回帰を実行（修正版）
results_fixed = perform_linear_regression_fixed(sample_data, explanatory_vars, dependent_var, min_data_count=50)

# 最初の結果のサマリーを表示（例）
first_key = list(results_fixed.keys())[0]
print(results_fixed[first_key].summary())


In [ ]:
# 結果をExcelに出力する関数（修正版）
def output_results_to_excel_fixed(results, file_name):
    # 新しいワークブックを作成
    wb = Workbook()
    
    # 各セクターごとにシートを作成
    for key in results.keys():
        date, sector = key
        model = results[key]
        
        # シート名を設定（セクター名と日付）
        sheet_name = f"{sector} {pd.Timestamp(date).strftime('%Y-%m')}"
        if sheet_name not in wb.sheetnames:
            ws = wb.create_sheet(title=sheet_name)
        else:
            ws = wb[sheet_name]

        # 結果をシートに書き込む
        ws.append(["Coefficients"])
        for param in model.params.index:
            ws.append([param, model.params[param]])

        ws.append([])
        ws.append(["P-values"])
        for param in model.pvalues.index:
            ws.append([param, model.pvalues[param]])

        ws.append([])
        ws.append(["R-squared", model.rsquared])
        ws.append(["Adj. R-squared", model.rsquared_adj])

    # ファイルに保存
    with BytesIO() as f:
        wb.save(f)
        f.seek(0)
        with open(file_name, 'wb') as out_file:
            out_file.write(f.read())

# 結果をExcelファイルに出力（修正版）
output_results_to_excel_fixed(results_fixed, "regression_results.xlsx")

"Excelファイル 'regression_results.xlsx' が生成されました。"
